In [ ]:
import os
import time
import copy
import numpy as np
import pandas as pd
from PIL import Image
import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,models,transforms
import torch.optim as optim
# from torchsummary import summary
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from torch.nn import Parameter

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import itertools

from joblib import Parallel, delayed

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import shutil
from scipy.fftpack import dct, idct

In [ ]:
# 读取train.txt
train_df = pd.read_csv('../input/covidxct/train_COVIDx_CT-3A.txt', sep=" ", header=None)
train_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
train_df=train_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

In [ ]:
# labels={0:'Normal',1:'Pneumonia',2:'COVID-19'}
train_df.head()
train_df.label.value_counts()

In [ ]:
image_path = '../input/covidxct/3A_images/'  #directory path
train_df['filename'] = image_path+train_df['filename']
train_df.shape

In [ ]:
train_df=train_df.reset_index()

In [ ]:
train_df.filename

In [ ]:
!mkdir -p /tmp/train_images_enhanced 

In [ ]:
def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')
def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')  
def dtc_transform(img):  #DCT-filter
    return idct2(dct2(img))

def white_balance(channel, perc = 0.05):   #White balance function
    mi, ma = (np.percentile(channel, perc), np.percentile(channel,100.0-perc))
    channel = np.uint8(np.clip((channel-mi)*255.0/(ma-mi), 0, 255))
    return channel

def clahe():     #CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    return clahe

def image_enhancer(directory_path,df):
    save_dir = '/tmp/train_images_enhanced'  
    for i in range(150000,200000):
        image_name = df.filename[i].split("/")[-1]
        image_path = os.path.join(directory_path, image_name)
        image = cv2.imread(image_path) 
        image = dtc_transform(image)     #DCT-filter to remove noise
        
        # White Balance
        image_WB  = np.dstack([white_balance(channel, 0.05) for channel in cv2.split(image)] )
        gray_image = cv2.cvtColor(image_WB, cv2.COLOR_RGB2GRAY)

        # CLAHE
        clahe_function = clahe()
        image_clahe = clahe_function.apply(gray_image)
        image = cv2.cvtColor(image_clahe, cv2.COLOR_GRAY2RGB) 
    
        cv2.imwrite(f'{save_dir}/{image_name}', image) 
    return

In [ ]:
train_images = "../input/crop-images/train_images/"
image_enhancer(train_images,train_df)

In [ ]:
shutil.make_archive(base_name='/kaggle/working/train_images_enhanced',
                    format='zip',
                    root_dir='/tmp/',
                    base_dir='train_images_enhanced')